In [1]:
from itertools import permutations
# creating list of all possible permutation

data_augmentations = ['GN', 'TT', 'GAN']
# GN = Random Gaussian Noise
# TT = Time Transformation
# GAN = GAN models

all_permutations = []
for r in range(0, len(data_augmentations)+1):
  perms = list(permutations(data_augmentations, r))
  all_permutations.extend(perms)

In [2]:
for p in all_permutations:
  print(p)

()
('GN',)
('TT',)
('GAN',)
('GN', 'TT')
('GN', 'GAN')
('TT', 'GN')
('TT', 'GAN')
('GAN', 'GN')
('GAN', 'TT')
('GN', 'TT', 'GAN')
('GN', 'GAN', 'TT')
('TT', 'GN', 'GAN')
('TT', 'GAN', 'GN')
('GAN', 'GN', 'TT')
('GAN', 'TT', 'GN')


In [3]:
# import data augmentation techniques
from EEGNoiseAndTimeTransformations import plot_eeg_signals, add_gaussian_noise, time_transformation

In [4]:
import pandas as pd
# import training data

# path to the training file pkl
train_df_path = '/Users/ryan/Desktop/StackedDataAugmentation/physionetdata/eegmmidb_train_df.pkl'
val_df_path = '/Users/ryan/Desktop/StackedDataAugmentation/physionetdata/eegmmidb_val_df.pkl'

In [5]:
from EEGMMIDBDatasetLoaderV2 import EEGMMIDBDataset
from torch.utils.data import DataLoader

train_dataset = EEGMMIDBDataset(pickle_path=train_df_path, purpose='eegnet', onehot=True)
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = EEGMMIDBDataset(pickle_path=val_df_path, purpose='eegnet', onehot=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [6]:
# print(len(train_loader))
# for X, y in train_loader:
#   print(f'Batch EEG Shape: {X.shape}')
#   print(f'Batch Labels: {y}')
#   break

In [7]:
# insall arl-eegmodels
# !git clone https://github.com/vlawhern/arl-eegmodels.git

In [8]:
!ls

Python(3099) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


CreateDatasetPermutations.ipynb      GANTraining
EEGMMIDBDatasetLoaderV2.ipynb        __pycache__
EEGMMIDBDatasetLoaderV2.py           arl-eegmodels
EEGNoiseAndTimeTransformations.ipynb physionetdata
EEGNoiseAndTimeTransformations.py


In [9]:
import numpy as np

def torch_dataset_to_numpy(dataset):
  """
  turns torch.tensors -> np.ndarrays
  """
  X_list, y_list = [],[]
  for i in range(len(dataset)):
    x, y = dataset[i]
    X_list.append(x.numpy())
    y_list.append(y.numpy())

  return np.stack(X_list), np.stack(y_list)

In [10]:
X_train, y_train = torch_dataset_to_numpy(train_dataset)
X_val, y_val = torch_dataset_to_numpy(val_dataset)

In [11]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

X_train shape: (13656, 64, 640, 1)
y_train shape: (13656, 5)
X_val shape: (2923, 64, 640, 1)
y_val shape: (2923, 5)


In [12]:
# moving to arl-eegmodels directory
%cd arl-eegmodels/

/Users/ryan/Desktop/StackedDataAugmentation/arl-eegmodels


/Users/ryan/Desktop/StackedDataAugmentation/.venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
from EEGModels import EEGNet
import torch
import torch.nn as nn

# creating EEGNet
model = EEGNet(nb_classes=5,    # num of classes
               Chans=64,       # num of channels
               Samples=640,     # seq_len
               )
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train,
          batch_size=32,
          epochs=20,
          validation_split=0.2,)
loss, acc = model.evaluate(X_val, y_val)
print(f'Val Loss: {loss}, Val Acc: {acc}')

/Users/ryan/Desktop/StackedDataAugmentation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ryan/Desktop/StackedDataAugmentation/.venv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
